# 2D Aggregates Simulation
A simulation of 2D that demonstrates random motion of bacteria cells.<br><br>
Each cell is within an aggregate, even if it is the only cell that is in that aggregate. This allows the aggregates to add cells through collision and birth. Additionally, aggregates can lose cells when cells simply break apart or when cells die. The goal of this simulation is to provide information on how the aggregation of bacteria (specifically *Pseudomonas A.*)

## Cell Level Properties
* Birth
* Death
* Growth Rate

In [1]:
mutable struct Cell
    x::Float64
    y::Float64
    prob_birth::Float64
    dist_from_root_x::Float64
    dist_from_root_y::Float64
end

## Cell Functions
- periodic_boundary(cell)
  - Loops the cell to the other side of the map when it crosses the boundaries

In [2]:
function periodic_boundary(cell::Cell)
    if cell.x > POS_LIM
        cell.x = NEG_LIM + 1
    elseif cell.x < NEG_LIM
        cell.x = POS_LIM - 1
    end
    if cell.y > POS_LIM
        cell.y = NEG_LIM + 1
    elseif cell.y < NEG_LIM
        cell.y = POS_LIM - 1
    end
end

periodic_boundary (generic function with 1 method)

In [3]:
function reflective_boundary(cell::Cell, dx, dy)
    if ((cell.x + dx) > POS_LIM)
        cell.x = POS_LIM - (dx + cell.x - POS_LIM)
    elseif ((cell.x + dx) < NEG_LIM)
        cell.x = NEG_LIM - (dx + cell.x - NEG_LIM)
    else
        cell.x += dx
    end
    if ((cell.y + dy) > POS_LIM)
        cell.y = POS_LIM - (dy + cell.y - POS_LIM)
    elseif ((cell.y + dy) < negLim)
        cell.x = NEG_LIM - (dy + cell.y - NEG_LIM)
    else
        cell.y += dy
    end
end

reflective_boundary (generic function with 1 method)

In [4]:
function generate_newcell(parent_cell::Cell)
    x_diff = 4 * rand() - 2
    y_diff = 4 * rand() - 2
    newcell = Cell(
        parent_cell.x + x_diff,
        parent_cell.y + y_diff,
        parent_cell.prob_birth,
        parent_cell.dist_from_root_x + x_diff,
        parent_cell.dist_from_root_y + y_diff
    )
    return newcell
end

generate_newcell (generic function with 1 method)

## Aggregate Level Properties
* Movement
* Color

In [5]:
import ColorTypes
mutable struct Aggregate
    cells::Set{Cell}
    rgb::ColorTypes.RGBA
    size::Int32
    root::Cell
end

## Aggregate Functions
- collision(aggregate1, aggregate2)
  - handles the collision between 2 aggregates
- add_cell(aggregate, cell)
  - adds the cell to the aggregate
- del_cell(aggregate, cell)
  - removes the cell from the aggregate
- merge_aggregates(aggregate1, aggregate2)
  - adds all of one aggregates cells to another aggregate
direction


In [6]:
function collision(aggregate::Aggregate, new_aggregate::Aggregate)
    for cell1 in aggregate.cells
        for cell2 in new_aggregate.cells
            if (sqrt((cell1.x - cell2.x)^2 + (cell1.y - cell2.y)^2) <= COLLISION_DISTANCE)
                return true
            end
        end
    end
    return false
end

collision (generic function with 1 method)

In [7]:
function add_cell(aggregate::Aggregate, cell::Cell)
    push!(aggregate.cells, cell)
    aggregate.size += 1
end

add_cell (generic function with 1 method)

In [8]:
function del_cell(aggregate::Aggregate, cell::Cell)
    delete!(aggregate.cells, cell)
    aggregate.size -= 1
end

del_cell (generic function with 1 method)

In [9]:
function merge_aggregates(Aggregate_Map::Set{Aggregate}, aggregate::Aggregate, new_aggregate::Aggregate)
    # Completes a union of the aggregate.cells sets
    union!(aggregate.cells, new_aggregate.cells)
    aggregate.size += length(new_aggregate.cells)
    delete!(Aggregate_Map, new_aggregate)
end

merge_aggregates (generic function with 1 method)

In [10]:
function break_apart(Aggregate_Map::Set{Aggregate}, aggregate::Aggregate, breakaway_cell::Cell)
    breakaway_dist_x = breakaway_cell.dist_from_root_x
    breakaway_dist_y = breakaway_cell.dist_from_root_y
    
    x_mov = 2
    if breakaway_dist_x < 0
        x_mov = -2
    end

    y_mov = 2
    if breakaway_dist_y < 0
        y_mov = -2
    end
        
    breakaway_cell.x += x_mov
    breakaway_cell.y += y_mov
    
    # Create new aggregate with the cell as root
    new_aggregate = Aggregate(
        Set{Cell}([breakaway_cell]),
        RGBA(rand(), rand(), rand(), 1),
        Int32(1),
        breakaway_cell
    )
    delete!(aggregate.cells, breakaway_cell)
                
    push!(Aggregate_Map, new_aggregate)

end

break_apart (generic function with 1 method)

## Lattice
We use this struct to keep track of nutrient and cell concentrations. So far, there are two main instances of the lattice class in the simulation. For more information, read about Fick's Second Law.
* nutrient_lattice is a 2D array that keeps track of the amount of nutrient available in each location
* cell_density_lattice is a 2D array that keeps track of the density of cells in each location. This is directly mappable to the nutrient lattice.
### Lattice Properties
    * mesh_size: the dimensions of the lattice are mesh_size x mesh_size
    * matrix: the matrix representing the lattice
    * diffusion_coefficient: proporitionality constant between the flux and the gradient of the concentration
    * diffusion_dt: the proportion of a diffusion time step with respect to the simulation time step (e.g a diffusion_dt of 0.1 means nutrient diffusion occurs 10 times per simulation time step)

In [11]:
mutable struct Lattice
    mesh_size::Int64
    matrix::Matrix{Float64}
    diffusion_coefficient::Int64
    diffusion_dt::Float64
end

## Lattice Functions
- initialize_nutrient(lattice)
  - randomly initializes the gradient of the lattice
- initalize_density_to_zero(lattice)
  - initializes the density lattice by setting all 
- show_lattice(lattice)
  - visualizes the lattice
- calc_diffusion(lattice)
  - calculates the calculates the new nutrient lattice after diffusion
- calc_lattice_coords(x, y)
  - calculates the coordinates of a cell within the density lattice

In [12]:
function initialize_nutrient(lattice::Lattice)
    # lattice.matrix = 10 .* rand(Float64, (lattice.mesh_size, lattice.mesh_size))
    lattice.matrix = 10 .+ zeros(Float64, (lattice.mesh_size, lattice.mesh_size))
end

initialize_nutrient (generic function with 1 method)

In [13]:
function initialize_density_to_zero(lattice::Lattice)
    lattice.matrix = zeros(Float64, (lattice.mesh_size, lattice.mesh_size))
end

initialize_density_to_zero (generic function with 1 method)

In [14]:
using Plots

# Not to scale yet
function show_lattice(lattice::Lattice)
    return heatmap(
        lattice.matrix',
        color=:auto,
        xlabel="Mesh X Index",
        ylabel="Mesh Y Index",
        c=:Purples,
        xlims=(0, lattice.mesh_size),
        ylims=(0, lattice.mesh_size),
        clim=(0, 10),
        aspect_ratio=:equal,
    )
    Colorbar(map[1, 2], pltobj, label = "Reverse sequential colormap")
    return map
end

show_lattice (generic function with 1 method)

In [15]:
function calculate_diffusion(lattice)
    y_flux = lattice.matrix[2:end, :] .- lattice.matrix[1:end-1, :]
    y_flux_padded = zeros(Float64, (lattice.mesh_size + 1, lattice.mesh_size))
    
    # Scale gradient at boundary to control inflow and outflow
    y_flux_padded[1,:] .+= (1  * OUTFLOW_GRADIENT)
    y_flux_padded[MESH_SIZE + 1,:] .+= (1 * INFLOW_GRADIENT)
    
    y_flux_padded[2:end-1, :] .= y_flux
    y_flux_padded *= -lattice.diffusion_coefficient
    y_flux_padded /= LATTICE_THICKNESS
        
    
    dC = (y_flux_padded[2:end, :] .- y_flux_padded[1:end-1, :])/(-LATTICE_THICKNESS)
    
    x_flux = lattice.matrix[:, 2:end] .- lattice.matrix[:, 1:end-1]
    x_flux_padded = zeros(Float64, (lattice.mesh_size, lattice.mesh_size + 1))
    
    # Scale gradient at boundary to control inflow and outflow
    x_flux_padded[:, 1] .+= (1  * OUTFLOW_GRADIENT)
    x_flux_padded[:, MESH_SIZE + 1] .+= (1 * INFLOW_GRADIENT)
    
    x_flux_padded[:, 2:end-1] .= x_flux
    x_flux_padded *= -lattice.diffusion_coefficient
    x_flux_padded /= LATTICE_THICKNESS
    
    # add flux gradient to rate of change
    dC = dC - ((x_flux_padded[:, 2:end] .- x_flux_padded[:, 1:end-1]) / LATTICE_THICKNESS)
    
    # calculate delta C by multiplying with delta t and add to old array
    lattice.matrix .+= dC*lattice.diffusion_dt
end

calculate_diffusion (generic function with 1 method)

In [16]:
function calculate_lattice_coords(x, y)
    x_coord = trunc(Int64, (x + abs(NEG_LIM)) / LATTICE_THICKNESS)
    if x_coord == 0
        x_coord += 1
    end
    y_coord = trunc(Int64, (y + abs(NEG_LIM)) / LATTICE_THICKNESS)
    if y_coord == 0
        y_coord += 1
    end
    # y_coord = ceil(Int64, (y + 0.0001 + abs(NEG_LIM)) / LATTICE_THICKNESS)
    # if y_coord > MESH_SIZE
    #     y_coord -= 1
    # end
    return (x_coord, y_coord)
end

calculate_lattice_coords (generic function with 1 method)

## Simulation Code

In [53]:
import Base.run
using ColorTypes
function run()
    db = connect_to_db()
    Aggregate_Map = Set{Aggregate}()
    for cell in 1:INITIAL_PCELLS
        growth_rate = SLOW_GROWTH_RATE
        newcell = nothing
        if rand() >= .5
            growth_rate = FAST_GROWTH_RATE
        end
        newcell = Cell(
            rand((NEG_LIM / 2, POS_LIM / 2)),
            rand((NEG_LIM / 2, POS_LIM / 2)),
            growth_rate,
            0,
            0
        )
        new_aggregate = Aggregate(
                Set{Cell}([newcell]),
                RGBA(rand(), rand(), rand(), 1),
                Int32(1),
                newcell
            )
        push!(Aggregate_Map, new_aggregate)
        for i in 1:AGGREGATE_SIZE-1
            child_cell = generate_newcell(newcell)
            add_cell(new_aggregate, child_cell)
        end
    end
    
    # Initialize the nutrient lattice
    nutrient_lattice = Lattice(MESH_SIZE, zeros(Float64, MESH_SIZE, MESH_SIZE),
        DIFFUSION_COEFFICIENT, DIFFUSION_DT)
    initialize_nutrient(nutrient_lattice)
    
    # Create and initialize the density map
    cell_density_lattice = Lattice(MESH_SIZE, zeros(Float64, MESH_SIZE, MESH_SIZE),
        DIFFUSION_COEFFICIENT, DIFFUSION_DT)
    # Updates density map based on initial cell conditions
    for aggregate in Aggregate_Map
        for cell in aggregate.cells
            x, y = calculate_lattice_coords(cell.x, cell.y)
            cell_density_lattice.matrix[x, y] += 1
        end
    end
    
    
    photo_file_list = []
    for i in 1:T_MAX
        photo_file = "graphs/" * string(i) * ".png"
        push!(photo_file_list, photo_file)
        
        cell_density_lattice_placeholder = cell_density_lattice
        
        for aggregate in Aggregate_Map
            move = rand() < PROB_MOVE # Decides whether or not the aggregate moves
            if move
                dir = rand() * 2 * π
                dx = 10 * cos(dir) # Change in y
                dy = 10 * sin(dir) # Change in x
            end
            new_cells = Set{Cell}() # These cells are added to aggregate during time step
            for cell in aggregate.cells
                if !(cell in new_cells) # Makes sure that a new cell does not reproduce
                    nutrient_x, nutrient_y = calculate_lattice_coords(cell.x, cell.y)
                    # Check if cell dies
                    if rand() < PROB_DEATH
                        del_cell(aggregate, cell)
                        if aggregate.size <= 0
                            delete!(Aggregate_Map, aggregate)
                        end
                        cell_density_lattice.matrix[nutrient_x, nutrient_y] -= 1
                        continue
                    end
                    # Check if cell breaks apart
                    break_probability = BREAK_PROBABILITY
                    # Aggregates tend to start shedding cells more at about 70 shells
                    if aggregate.size >= 70
                        break_probability += .01 * maximum(10, aggregate.size - 69)
                    end
                    if aggregate.size > 1 && rand() < break_probability
                        break_apart(Aggregate_Map, aggregate, cell)
                        if aggregate.size <= 0
                            delete!(Aggregate_Map, aggregate)
                        end
                        continue
                    end

                    # Move cell
                    if move
                        cell.x += dx
                        cell.y += dy
                        cell_density_lattice.matrix[nutrient_x, nutrient_y] -= 1
                        # Check boundary condition
                        if cell.x > POS_LIM || cell.x < NEG_LIM ||
                            cell.y > POS_LIM || cell.y < NEG_LIM
                            # User can set the boundary condition which determines how a cell
                            # that has exited the boundaries is handled
                            if BOUNDARY_CONDITION == PERIODIC::BoundaryConditions
                                periodic_boundary(cell)
                            elseif BOUNDARY_CONDITION == OUTFLOW::BoundaryConditions
                                del_cell(aggregate, cell)
                                if aggregate.size <= 0
                                    delete!(Aggregate_Map, aggregate)
                                        end
                                continue
                            elseif BOUNDARY_COUNDITION == REFLECTIVE::BoundaryConditions
                                reflective_boundary(cell, dx, dy)
                            end
                        end
                        new_nutrient_x, new_nutrient_y = calculate_lattice_coords(cell.x, cell.y)
                        cell_density_lattice.matrix[new_nutrient_x, new_nutrient_y] += 1
                    end

                    
                    if nutrient_lattice.matrix[nutrient_x, nutrient_y] < 0
                        nutrient_lattice.matrix[nutrient_x, nutrient_y] = 0
                    end
                    
                    # Use P(N -> N+1) formula for growth probability
                    cell.prob_birth = MAX_GROWTH_RATE * (nutrient_lattice.matrix[nutrient_x, nutrient_y] /
                        (HALF_MAXIMAL_CONCENTRATION + nutrient_lattice.matrix[nutrient_x, nutrient_y])) * SIM_DT
                    
                    if rand() < cell.prob_birth
                        newcell = generate_newcell(cell)

                        if rand() < CREATES_OWN_AGG_PROBABILITY
                            # Create new aggregate for newcell
                            push!(
                                Aggregate_Map, 
                                Aggregate(
                                    Set([newcell]),
                                    RGBA(rand(), rand(), rand(), 1),
                                    Int32(1),
                                    newcell
                                )
                            )
                        else
                            add_cell(aggregate, newcell)
                        end
                        push!(new_cells, newcell)
                        new_cell_nutrient_x, new_cell_nutrient_y = 
                            calculate_lattice_coords(newcell.x, newcell.y)
                        cell_density_lattice.matrix[new_cell_nutrient_x, new_cell_nutrient_y] += 1
                    end
                end
            end

            if aggregate.size <= 0
                delete!(Aggregate_Map, aggregate)
                break
            else
                for aggregate2 in Aggregate_Map
                    if aggregate2 != aggregate
                        if collision(aggregate, aggregate2)
                            if rand() < MERGE_ON_COLLISION_PROBABILITY
                                if aggregate.size >= aggregate2.size
                                    merge_aggregates(Aggregate_Map, aggregate, aggregate2)
                                else
                                    merge_aggregates(Aggregate_Map, aggregate2, aggregate)
                                end
                            end
                        end
                    end
                end
            end
        end
        
        if GENERATE_GRAPHS
            cell_movement = plot_agg(Aggregate_Map, photo_file, i)
            nutrient_graph = show_lattice(nutrient_lattice)
            create_plots(cell_movement, nutrient_graph, photo_file)
        end
        if WRITE_TO_DB
            write_to_db(Aggregate_Map, db, i)
        end
        
        # Update resource (Use cell_density_placeholder to update lattice)
        nutrient_time_steps = 1 / DIFFUSION_DT
        for j in 1:nutrient_time_steps
            calculate_diffusion(nutrient_lattice)
        end
        nutrient_lattice.matrix .-= (1 / YIELD_COEFFICIENT) * MAX_GROWTH_RATE * SIM_DT .*
            (nutrient_lattice.matrix ./ (HALF_MAXIMAL_CONCENTRATION .+ nutrient_lattice.matrix)) .*
            cell_density_lattice_placeholder.matrix
    end
end

run (generic function with 2 methods)

In [54]:
using Plots

function plot_agg(Aggregate_Map::Set{Aggregate}, photo_file::String, CURR_T::Int64)  
    p = Plots.scatter(legend=false, xlabel="X Microns", ylabel="Y Microns", hover=true, aspect_ratio=:equal)
    title!("Discrete Time: $CURR_T seconds")
    xlims!(NEG_LIM, POS_LIM)
    ylims!(NEG_LIM, POS_LIM)
    for aggregate in Aggregate_Map
        for cell in aggregate.cells
            Plots.scatter!(p, [cell.x], [cell.y], color=aggregate.rgb;)
        end
    end
    return p
    # Plots.savefig(photo_file)
end


plot_agg (generic function with 1 method)

In [55]:
using Plots

function create_plots(p1,p2, photo_file)
    plot(p1,p2)
    Plots.savefig(photo_file)
end

create_plots (generic function with 1 method)

## Database Code

In [56]:
import Pkg
using SQLite
using DataFrames

function connect_to_db()
    if WRITE_TO_DB
        db = SQLite.DB(DB_NAME)
        query = "CREATE TABLE IF NOT EXISTS "
        query = query * TABLE_NAME
        query = query * """ (
                key INTEGER PRIMARY KEY AUTOINCREMENT,
                time_step INTEGER,
                aggregate_size INTEGER,
                number_of_aggregates INTEGER
            )"""
        SQLite.execute(db, query)
        return db
    else
        return nothing
    end
end

function write_to_db(Aggregate_Map::Set{Aggregate}, db::SQLite.DB, i::Int64)
    for aggregate in Aggregate_Map
        try
            # Check if a record exists with the specified conditions
            agg_size = aggregate.size
            query = SQLite.DBInterface.execute(db, "SELECT aggregate_size FROM $TABLE_NAME
                WHERE aggregate_size = ? AND time_step = ? LIMIT 1", (agg_size, i))
            df = DataFrames.DataFrame(query)

            if nrow(df) != 0
                # Update the existing record
                SQLite.execute(db, "UPDATE $TABLE_NAME SET number_of_aggregates = number_of_aggregates + 1
                    WHERE aggregate_size = ? AND time_step = ?", (agg_size, i))
            else
                # Insert a new record
                SQLite.execute(db, "INSERT INTO $TABLE_NAME (aggregate_size, time_step, number_of_aggregates)
                    VALUES (?, ?, 1)", (agg_size, i))
            end
        catch ex
            # Insert a new record
            println("SQLITE ERROR: $ex")
        end
    end
end

write_to_db (generic function with 1 method)

## Run the simuliation from here:

Choose whether you would like to save the simulation data to a databse and whether you would like to display the output as graphs. If you would like to save the images of the graphs as a gif, open terminal/command line and type 
```
cd IBMAggregateModeling
```
```
python3 graphs/make_gif.py
```
to create a gif.
This enum allows the user to choose what type of boundary condition they would like to use.\nThis enum allows the user to choose what type of boundary condition they would like to use.\nThe choices are:
- Periodic:
  - Each cell that crosses boundary loops over to other side of map
- Outflow:
  - Each cell the crosses boundary is deleted from the aggregate (cell death)

In [57]:
############################### SETUP ####################################
@enum BoundaryConditions PERIODIC=1 OUTFLOW REFLECTIVE
##########################################################################

In [58]:
######################## SIMULATION PARAMETERS #############################
T_MAX = 100 # Number of time steps
INITIAL_PCELLS = 7 # Initial number of planktonic cells
PROB_MOVE = 0.9 # Probability the cell moves in any given iteration
PROB_DEATH = 0.05 # Probability cell dies
NEG_LIM = -150 # Boundaries of space
POS_LIM = 150  # Boundaries of space
SLOW_GROWTH_RATE = .15
FAST_GROWTH_RATE = .3
MAX_GROWTH_RATE = 1.5
BREAK_PROBABILITY = 0.001
MERGE_ON_COLLISION_PROBABILITY = 0.1
AGGREGATE_SIZE = 5 # Original aggregate size
CREATES_OWN_AGG_PROBABILITY = 0
BOUNDARY_CONDITION = OUTFLOW::BoundaryConditions
GENERATE_GRAPHS = true
WRITE_TO_DB = false
DB_NAME = "test.db"
TABLE_NAME = "test1"
COLLISION_DISTANCE = 2

# Lattice variables
YIELD_COEFFICIENT = 1
HALF_MAXIMAL_CONCENTRATION = 0.3
MESH_SIZE = 10 # lattice dimensions = (meshSize x meshSize)
LATTICE_THICKNESS = (POS_LIM - NEG_LIM) / MESH_SIZE # thickness of each layer (uniform)
DIFFUSION_COEFFICIENT = 15 # micrometer squared
DIFFUSION_DT = .1
SIM_DT = 1
INFLOW_GRADIENT = 10
OUTFLOW_GRADIENT = 1
############################################################################

run()

## Setup for racing and comparing speed/functionality of julia sim to python sim

In [63]:
    ######################## SIMULATION PARAMETERS #############################
    T_MAX = 40 # Number of time steps
    INITIAL_PCELLS = 3 # Initial number of planktonic cells
    PROB_DEATH = 0.05 # Probability cell dies
    PROB_MOVE = 0.9 # Probability the cell moves in any given iteration
    NEG_LIM = -150 # Boundaries of space
    POS_LIM = 150  # Boundaries of space
    SLOW_GROWTH_RATE = .1
    FAST_GROWTH_RATE = .2
    BREAK_PROBABILITY = 0
    AGGREGATE_SIZE = 2 # Original aggregate size
    CREATES_OWN_AGG_PROBABILITY = 0.5
    BOUNDARY_CONDITION = OUTFLOW::BoundaryConditions
    GENERATE_GRAPHS = true
    WRITE_TO_DB = false
    DB_NAME = "race_python_v_julia/julia1.db"
    ############################################################################
    
    # @time begin run() end
    function race()
        for i in 1:100
            run()
        end
    end
    @time begin race() end

LoadError: MethodError: objects of type Int64 are not callable
Maybe you forgot to use an operator such as [36m*, ^, %, / etc. [39m?

In [66]:
using Plots
using ColorTypes
p = Plots.scatter(legend=false, xlabel="X Microns", ylabel="Y Microns")
title!("Discrete Time Simulation: $CURR_T seconds")
xlims!(NEG_LIM, POS_LIM)
ylims!(NEG_LIM, POS_LIM)
custom_color = RGBA(0.5, 0.8, 0.8, 1)
Plots.scatter!(p, [1], [2], color=custom_color;)
Plots.scatter!(p, [3], [4], color=custom_color;)
Plots.scatter!(p, [5], [1]; color=custom_color)
Plots.savefig("my_plot.png")

LoadError: UndefVarError: `CURR_T` not defined

## Julia Experiment 1
- Run with 5 initial aggregates of 1 cell each
- Probability of death is 0.03
- No new aggregates can be created and break probability is 0


In [54]:
######################## SIMULATION PARAMETERS #############################
T_MAX = 100 # Number of time steps
INITIAL_PCELLS = 5 # Initial number of planktonic cells
PROB_MOVE = 0.9 # Probability the cell moves in any given iteration
PROB_DEATH = 0.03 # Probability cell dies
NEG_LIM = -150 # Boundaries of space
POS_LIM = 150  # Boundaries of space
SLOW_GROWTH_RATE = .15
FAST_GROWTH_RATE = .3
MAX_GROWTH_RATE = 1.5
BREAK_PROBABILITY = 0
AGGREGATE_SIZE = 1 # Original aggregate size
CREATES_OWN_AGG_PROBABILITY = 0
BOUNDARY_CONDITION = OUTFLOW::BoundaryConditions
GENERATE_GRAPHS = true
WRITE_TO_DB = false
DB_NAME = "experiments/JuliaExperiment1/results.db"
TABLE_NAME = "RUN_"
COLLISION_DISTANCE = 2

# Lattice variables
YIELD_COEFFICIENT = 4
HALF_MAXIMAL_CONCENTRATION = 0.3
MESH_SIZE = 10 # lattice dimensions = (meshSize x meshSize)
LATTICE_THICKNESS = (POS_LIM - NEG_LIM) / MESH_SIZE # thickness of each layer (uniform)
DIFFUSION_COEFFICIENT = 15 # micrometer squared
DIFFUSION_DT = .1
SIM_DT = 1
INFLOW_GRADIENT = 10
OUTFLOW_GRADIENT = 1
############################################################################

for i in 1:100
    if i > 10 && i < 100
        TABLE_NAME = TABLE_NAME[1:end-2] * string(i)
    elseif i > 100 && i < 1000
        TABLE_NAME = TABLE_NAME[1:end-3] * string(i)
    else
        TABLE_NAME = TABLE_NAME[1:end-1] * string(i)
    end
    run()
end

LoadError: SQLiteException("file is not a database")